### Train and test datasets


In [ ]:
import numpy as np
import pandas as pd

# Packages for train/test dataset split
from sklearn.model_selection import train_test_split

In [ ]:
# Add Google Drive as an accessible path (Optional if you are running from Jupyter Notebook)
from google.colab import drive
drive.mount('/content/drive')

# change path to the designated google drive folder
# otherwise, data will be saved in /content folder which you may have issue locating
%cd /content/drive/My Drive/Colab Notebooks/SMT203 Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/SMT203 Project


In [ ]:
mypath = '.'

In [ ]:
df2021 = pd.read_csv(f'{mypath}/lgbt_reddit_cleaned_labelled_2021.csv', encoding= 'unicode_escape') #csv data to read

In [ ]:
count_nan_in_df = df2021.isnull().sum()
print (count_nan_in_df)
count_nan = df2021['clean_text_1'].isna().sum()
print(count_nan)

title              1
user              40
body               1
Sentiment       1069
score              1
created            1
timestamp          1
translated         1
clean_text_1       4
clean_text_2       7
clean_text_3       7
dtype: int64
4


In [ ]:
df2021 = df2021.dropna(subset=['clean_text_1'])
df2021


,title,user,body,Sentiment,score,created,timestamp,translated,clean_text_1,clean_text_2,clean_text_3
0,Minister Shanmugam's statement on retaining 377A,didijxk,Kicking the can down the road for a future PM ...,Neutral,23.0,1.619328e+09,2021-04-25 05:17:30,Kicking the can down the road for a future PM ...,kicking the can down the road for a future pm ...,kicking road future pm handle,kicking road future pm handle
1,Minister Shanmugam's statement on retaining 377A,CrunchyleaveOO,Many on this sub likes to think that itâs ju...,Neutral,811.0,1.619318e+09,2021-04-25 02:38:58,Many on this sub likes to think that itâs ju...,many on this sub likes to think that it s just...,many sub likes think religious conservative pe...,many sub likes think religious conservative pe...
2,Minister Shanmugam's statement on retaining 377A,abuqaboom,How things change in 7 years: https://www.toda...,Neutral,165.0,1.619319e+09,2021-04-25 02:43:53,How things change in 7 years: https://www.toda...,how things change in years,things change years,things change years
3,Minister Shanmugam's statement on retaining 377A,Deminovia,Although im in favour of repealing 377A becaus...,Positive,116.0,1.619328e+09,2021-04-25 05:24:08,Although im in favour of repealing 377A becaus...,although im in favour of repealing because i ...,although im favour repealing feel criminalisin...,although im favour repealing feel criminalisin...
4,Minister Shanmugam's statement on retaining 377A,paper_boxes,Was looking through the 2007 parliamentary deb...,Neutral,25.0,1.619335e+09,2021-04-25 07:11:40,Was looking through the 2007 parliamentary deb...,was looking through the parliamentary debates...,looking parliamentary debates penal code amend...,looking parliamentary debates penal code amend...
...,...,...,...,...,...,...,...,...,...,...,...
1331,Joint Statement by Asian NGOs to condemn arres...,drmchsr0,> \n> I believe that truth is objective. There...,NaN,2.0,1.611983e+09,2021-01-30 05:09:28,Those be called facts to me. Objective reality...,those be called facts to me objective reality ...,called facts objective reality fact facts care...,called facts objective reality fact facts care...
1332,Joint Statement by Asian NGOs to condemn arres...,Windreon,">Freedom of speech, assembly and association\n...",NaN,3.0,1.611991e+09,2021-01-30 07:13:14,Example: Pink Dot. \n\nYou are free to disagre...,example pink dot you are free to disagree th...,example pink dot free disagree idea peaceful p...,example free disagree idea peaceful protest w...
1333,Joint Statement by Asian NGOs to condemn arres...,Pyrrylanion,It is a good politicianâs job to be calculat...,NaN,3.0,1.612018e+09,2021-01-30 14:44:19,It is a good politicianâs job to be calculat...,it is a good politician s job to be calculativ...,good politician job calculative lgbt kids vote...,good politician job calculative kids vote fir...
1334,Joint Statement by Asian NGOs to condemn arres...,MagosBiologis,> Those be called facts to me. Objective reali...,NaN,0.0,1.611990e+09,2021-01-30 06:53:17,"Uhh ""fact"" and ""truth"" are interchangeable. \n...",uhh fact and truth are interchangeable if so...,uhh fact truth interchangeable something true ...,uhh fact truth interchangeable something true ...


In [ ]:
count_nan_in_df = df2021.isnull().sum()
print (count_nan_in_df)

title              0
user              39
body               0
Sentiment       1065
score              0
created            0
timestamp          0
translated         0
clean_text_1       0
clean_text_2       3
clean_text_3       3
dtype: int64


In [ ]:
df2021

,title,user,body,Sentiment,score,created,timestamp,translated,clean_text_1,clean_text_2,clean_text_3
0,Minister Shanmugam's statement on retaining 377A,didijxk,Kicking the can down the road for a future PM ...,Neutral,23.0,1.619328e+09,2021-04-25 05:17:30,Kicking the can down the road for a future PM ...,kicking the can down the road for a future pm ...,kicking road future pm handle,kicking road future pm handle
1,Minister Shanmugam's statement on retaining 377A,CrunchyleaveOO,Many on this sub likes to think that itâs ju...,Neutral,811.0,1.619318e+09,2021-04-25 02:38:58,Many on this sub likes to think that itâs ju...,many on this sub likes to think that it s just...,many sub likes think religious conservative pe...,many sub likes think religious conservative pe...
2,Minister Shanmugam's statement on retaining 377A,abuqaboom,How things change in 7 years: https://www.toda...,Neutral,165.0,1.619319e+09,2021-04-25 02:43:53,How things change in 7 years: https://www.toda...,how things change in years,things change years,things change years
3,Minister Shanmugam's statement on retaining 377A,Deminovia,Although im in favour of repealing 377A becaus...,Positive,116.0,1.619328e+09,2021-04-25 05:24:08,Although im in favour of repealing 377A becaus...,although im in favour of repealing because i ...,although im favour repealing feel criminalisin...,although im favour repealing feel criminalisin...
4,Minister Shanmugam's statement on retaining 377A,paper_boxes,Was looking through the 2007 parliamentary deb...,Neutral,25.0,1.619335e+09,2021-04-25 07:11:40,Was looking through the 2007 parliamentary deb...,was looking through the parliamentary debates...,looking parliamentary debates penal code amend...,looking parliamentary debates penal code amend...
...,...,...,...,...,...,...,...,...,...,...,...
1331,Joint Statement by Asian NGOs to condemn arres...,drmchsr0,> \n> I believe that truth is objective. There...,NaN,2.0,1.611983e+09,2021-01-30 05:09:28,Those be called facts to me. Objective reality...,those be called facts to me objective reality ...,called facts objective reality fact facts care...,called facts objective reality fact facts care...
1332,Joint Statement by Asian NGOs to condemn arres...,Windreon,">Freedom of speech, assembly and association\n...",NaN,3.0,1.611991e+09,2021-01-30 07:13:14,Example: Pink Dot. \n\nYou are free to disagre...,example pink dot you are free to disagree th...,example pink dot free disagree idea peaceful p...,example free disagree idea peaceful protest w...
1333,Joint Statement by Asian NGOs to condemn arres...,Pyrrylanion,It is a good politicianâs job to be calculat...,NaN,3.0,1.612018e+09,2021-01-30 14:44:19,It is a good politicianâs job to be calculat...,it is a good politician s job to be calculativ...,good politician job calculative lgbt kids vote...,good politician job calculative kids vote fir...
1334,Joint Statement by Asian NGOs to condemn arres...,MagosBiologis,> Those be called facts to me. Objective reali...,NaN,0.0,1.611990e+09,2021-01-30 06:53:17,"Uhh ""fact"" and ""truth"" are interchangeable. \n...",uhh fact and truth are interchangeable if so...,uhh fact truth interchangeable something true ...,uhh fact truth interchangeable something true ...


In [ ]:
df2021['Sentiment'].value_counts()

Neutral     158
Negative     92
Positive     17
Name: Sentiment, dtype: int64

In [ ]:
df2021['Sentiment'].head()

0     Neutral
1     Neutral
2     Neutral
3    Positive
4     Neutral
Name: Sentiment, dtype: object

In [ ]:
!pip install datasets transformers[sentencepiece]

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
from transformers import pipeline

In [ ]:
df2021.head(268)

,title,user,body,Sentiment,score,created,timestamp,translated,clean_text_1,clean_text_2,clean_text_3
0,Minister Shanmugam's statement on retaining 377A,didijxk,Kicking the can down the road for a future PM ...,Neutral,23.0,1.619328e+09,2021-04-25 05:17:30,Kicking the can down the road for a future PM ...,kicking the can down the road for a future pm ...,kicking road future pm handle,kicking road future pm handle
1,Minister Shanmugam's statement on retaining 377A,CrunchyleaveOO,Many on this sub likes to think that itâs ju...,Neutral,811.0,1.619318e+09,2021-04-25 02:38:58,Many on this sub likes to think that itâs ju...,many on this sub likes to think that it s just...,many sub likes think religious conservative pe...,many sub likes think religious conservative pe...
2,Minister Shanmugam's statement on retaining 377A,abuqaboom,How things change in 7 years: https://www.toda...,Neutral,165.0,1.619319e+09,2021-04-25 02:43:53,How things change in 7 years: https://www.toda...,how things change in years,things change years,things change years
3,Minister Shanmugam's statement on retaining 377A,Deminovia,Although im in favour of repealing 377A becaus...,Positive,116.0,1.619328e+09,2021-04-25 05:24:08,Although im in favour of repealing 377A becaus...,although im in favour of repealing because i ...,although im favour repealing feel criminalisin...,although im favour repealing feel criminalisin...
4,Minister Shanmugam's statement on retaining 377A,paper_boxes,Was looking through the 2007 parliamentary deb...,Neutral,25.0,1.619335e+09,2021-04-25 07:11:40,Was looking through the 2007 parliamentary deb...,was looking through the parliamentary debates...,looking parliamentary debates penal code amend...,looking parliamentary debates penal code amend...
...,...,...,...,...,...,...,...,...,...,...,...
263,Minister Shanmugam's statement on retaining 377A,ilkless,"hello, i see you have popped out of the woodwo...",Negative,3.0,1.619349e+09,2021-04-25 11:01:51,"hello, i see you have popped out of the woodwo...",hello i see you have popped out of the woodwor...,hello see popped woodwork holding account clea...,hello see popped woodwork holding account clea...
264,Minister Shanmugam's statement on retaining 377A,pingmr,>I do however always question why people in th...,Neutral,1.0,1.619406e+09,2021-04-26 03:04:23,Well it's because each time the Government rep...,well it is because each time the government re...,well time government repeats usual nonstatemen...,well time government repeats usual nonstatemen...
265,Minister Shanmugam's statement on retaining 377A,elpipita20,It also means the treatment of marginalised gr...,Negative,4.0,1.619352e+09,2021-04-25 11:56:09,It also means the treatment of marginalised gr...,it also means the treatment of marginalised gr...,also means treatment marginalised groups benev...,also means treatment marginalised groups benev...
266,Minister Shanmugam's statement on retaining 377A,Varantain,I wonder if PM told ~~his former personal lawy...,Neutral,4.0,1.619325e+09,2021-04-25 04:28:07,I wonder if PM told ~~his former personal lawy...,i wonder if pm told his former personal lawyer...,wonder pm told former personal lawyer ag fall ...,wonder pm told former personal lawyer ag fall ...


In [ ]:
sentences = list(df2021['clean_text_1'].iloc[0:266].values)
y_str = list(df2021['Sentiment'].iloc[0:266].values)

In [ ]:
pred_sentences = list(df2021['clean_text_1'].values)

In [ ]:
len(sentences)

266

In [ ]:
len(pred_sentences)

1332

In [ ]:
y = []
for each in y_str:
    if each == "Positive":
        y.append(0)
    elif each == "Negative":
        y.append(1)
    elif each == "Neutral":
        y.append(2)        
print(y_str[0], y[0])

Neutral 2


In [ ]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, random_state=1336)

In [ ]:
len(sentences_train)

212

In [ ]:
len(sentences_test)

54

In [ ]:
sentences_train, sentences_val, y_train, y_val = train_test_split(sentences_train, y_train, test_size=.2, random_state=1336)

###Tokenization

Alright, we've read in our dataset. Now let's tackle tokenization. We'll eventually train a classifier using
pre-trained DistilBert, so let's use the DistilBert tokenizer.

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(sentences_train, truncation=True, padding=True)
val_encodings = tokenizer(sentences_val, truncation=True, padding=True)
test_encodings = tokenizer(sentences_test, truncation=True, padding=True)

In [ ]:
import torch

class myDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)




In [ ]:
train_dataset = myDataset(train_encodings, y_train)
val_dataset = myDataset(val_encodings, y_val)
test_dataset = myDataset(test_encodings, y_test)

## Train


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=40,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



In [ ]:
# if it's not a binary classification, num_labels should be given! 
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

Step,Training Loss
10,1.053600
20,1.043400
30,1.021900
40,0.990000
50,0.962000
60,0.911500
70,0.876600
80,0.874600
90,0.785300
100,0.818700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=440, training_loss=0.29882320461117406, metrics={'train_runtime': 482.1224, 'train_samples_per_second': 14.021, 'train_steps_per_second': 0.913, 'total_flos': 750327355000080.0, 'train_loss': 0.29882320461117406, 'epoch': 40.0})

## Test

Now, let's evaluate the model performance.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

***** Running Evaluation *****
  Num examples = 54
  Batch size = 64


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_accuracy': 0.5555555555555556,
 'eval_f1': 0.32932060329320606,
 'eval_loss': 2.7461512088775635,
 'eval_precision': 0.3234649122807018,
 'eval_recall': 0.3361344537815126,
 'eval_runtime': 0.7076,
 'eval_samples_per_second': 76.313,
 'eval_steps_per_second': 1.413}

## Save & Load the model


In [ ]:
trainer.save_model()

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin


In [ ]:
new_model = DistilBertForSequenceClassification.from_pretrained("./results", num_labels=3)

new_trainer = Trainer(
    model=new_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

loading configuration file ./results/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading weights file ./results/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the

## Prediction

You can predict the labels of your unlabelled data. 

We will use the sentences in the test data (`sentences_test`) and predict the aspect labels. Here, we will pretend that they don't have any true labels! 



In [ ]:
pred_sentences[0]

'kicking the can down the road for a future pm to handle'

In [ ]:
len(pred_sentences)

1332

In [ ]:
# cleaned_pred_sentences = [x for x in pred_sentences if type(x) ==str]

In [ ]:
# create dataset for prediction
new_encodings = tokenizer(pred_sentences, truncation=True, padding=True)
# create dummy labels with the number of sentences to predict. 
y_new = np.full(len(pred_sentences), 1)
new_dataset = myDataset(new_encodings, y_new)

In [ ]:
new_predictions = new_trainer.predict(new_dataset)

***** Running Prediction *****
  Num examples = 1332
  Batch size = 64


In [ ]:
new_preds = np.argmax(new_predictions.predictions, axis=-1)
sump = 0
sumn = 0
sumneg = 0
for i in new_preds:
  if i == 0:
    sump += 1
  elif i == 1:
    sumneg += 1
  else:
    sumn += 1
print(f'There are {sump} positive comments, {sumneg} negative comments and {sumn} neutral comments')

There are 25 positive comments, 461 negative comments and 846 neutral comments


In [ ]:
# count_nan_in_df = df2021.isnull().sum()
# print (count_nan_in_df)



0


In [ ]:
# df2021 = df2021.dropna(subset=['clean_text_1'])
# print(df2021)

TypeError: ignored

In [ ]:
# count_nan_in_df = df2021.isnull().sum()
# print (count_nan_in_df)

In [ ]:
df2021['Prediction'] = new_preds.tolist()

In [ ]:
def add_predictions(pred):
  if pred == 0:
    pred = 'Positive'
  elif pred == 1:
    pred = 'Negative'
  else:
    pred = 'Neutral'
  return pred 

In [ ]:
df2021['Prediction'] = df2021['Prediction'].apply(add_predictions)
df2021.head()

,title,user,body,Sentiment,score,created,timestamp,translated,clean_text_1,clean_text_2,clean_text_3,Prediction
0,Minister Shanmugam's statement on retaining 377A,didijxk,Kicking the can down the road for a future PM ...,Neutral,23.0,1.619328e+09,2021-04-25 05:17:30,Kicking the can down the road for a future PM ...,kicking the can down the road for a future pm ...,kicking road future pm handle,kicking road future pm handle,Neutral
1,Minister Shanmugam's statement on retaining 377A,CrunchyleaveOO,Many on this sub likes to think that itâs ju...,Neutral,811.0,1.619318e+09,2021-04-25 02:38:58,Many on this sub likes to think that itâs ju...,many on this sub likes to think that it s just...,many sub likes think religious conservative pe...,many sub likes think religious conservative pe...,Neutral
2,Minister Shanmugam's statement on retaining 377A,abuqaboom,How things change in 7 years: https://www.toda...,Neutral,165.0,1.619319e+09,2021-04-25 02:43:53,How things change in 7 years: https://www.toda...,how things change in years,things change years,things change years,Neutral
3,Minister Shanmugam's statement on retaining 377A,Deminovia,Although im in favour of repealing 377A becaus...,Positive,116.0,1.619328e+09,2021-04-25 05:24:08,Although im in favour of repealing 377A becaus...,although im in favour of repealing because i ...,although im favour repealing feel criminalisin...,although im favour repealing feel criminalisin...,Positive
4,Minister Shanmugam's statement on retaining 377A,paper_boxes,Was looking through the 2007 parliamentary deb...,Neutral,25.0,1.619335e+09,2021-04-25 07:11:40,Was looking through the 2007 parliamentary deb...,was looking through the parliamentary debates...,looking parliamentary debates penal code amend...,looking parliamentary debates penal code amend...,Neutral


In [ ]:
df2021['Prediction'].value_counts()

Neutral     846
Negative    461
Positive     25
Name: Prediction, dtype: int64

In [ ]:
df2021.to_csv(f'{mypath}/lgbt_reddit_cleaned_classification_2021.csv', index=False)